In [28]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
)
import pandas as pd

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
#### Loading directories ####
npz_path = "/home/tharushi/Desktop/hackathon/10061/patches_128/real_plus_fft_dataset_128.npz"
data = np.load(npz_path)
X = data["X"]
y = data["y"]

print("X:", X.shape, X.dtype)  # (N,2,128,128)
print("y:", y.shape, y.dtype) 

X: (4000, 2, 128, 128) float32
y: (4000,) int64


In [ ]:
#### Dataset wrapper for NPZ-stored FFT features ####
class NPZDataset(Dataset):
    def __init__(self, npz_path):
        d = np.load(npz_path)
        self.X = d["X"].astype(np.float32)
        self.y = d["y"].astype(np.int64)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx]), torch.tensor(self.y[idx], dtype=torch.long)

dataset = NPZDataset(npz_path)
print("dataset size:", len(dataset), "pos rate:", dataset.y.mean())


dataset size: 4000 pos rate: 0.5


In [ ]:
# FFT-based CNN model for binary ice vs non-ice classification
class IceCNN_FFT(nn.Module):
    def __init__(self, in_channels=2):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),   # 128 -> 64

            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),   # 64 -> 32

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),   # 32 -> 16

            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.classifier = nn.Linear(128, 2)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

model = IceCNN_FFT(in_channels=X.shape[1]).to(DEVICE)


In [ ]:
### Construcing data loaders
indices = list(range(len(dataset)))

train_idx, val_idx = train_test_split(
    indices, test_size=0.2, stratify=dataset.y, random_state=42
)

train_ds = torch.utils.data.Subset(dataset, train_idx)
val_ds   = torch.utils.data.Subset(dataset, val_idx)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=64, shuffle=False)

# sanity check batch shapes
xb, yb = next(iter(train_loader))
print("batch X:", xb.shape, "batch y:", yb.shape)  


batch X: torch.Size([64, 2, 128, 128]) batch y: torch.Size([64])


In [ ]:
## Training and validation
EPOCHS = 5
LR = 1e-3
history = [] 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0
    total = 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        logits = model(imgs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * imgs.size(0)
        total += imgs.size(0)

    avg_loss = total_loss / total

    # validation
    model.eval()
    val_preds, val_true = [], []
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            logits = model(imgs)
            preds = logits.argmax(dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_true.extend(labels.cpu().numpy())
    
    metrics = {
    "epoch": epoch,
    "val_accuracy": accuracy_score(val_true, val_preds),
    "val_precision": precision_score(val_true, val_preds, pos_label=1, zero_division=0),
    "val_recall": recall_score(val_true, val_preds, pos_label=1, zero_division=0),
    "val_f1": f1_score(val_true, val_preds, pos_label=1, zero_division=0),
    "val_mcc": matthews_corrcoef(val_true, val_preds),
}

history.append(metrics)
df = pd.DataFrame(history)
df.to_csv("/home/tharushi/Desktop/hackathon/10061/patches_128/validation_metrics.csv", index=False)